<a href="https://colab.research.google.com/github/a42437-ml/master_thesis_files/blob/main/sip_features_tensorflow_pipeline_combined_detection_tc_06ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from tensorflow.keras.models import load_model

# Load model and files
model = load_model("covert_detector_model_2500_500_3.h5")
df = pd.read_csv("new_sip_features_tc-06_112-40.csv")  # Your new data
#expected_cols = pd.read_csv("training_columns.csv", header=None)[0].tolist()
# Correct if each column name is one row (as expected)
expected_cols = pd.read_csv("training_columns (3).csv", header=None).squeeze().tolist()

print(f"✅ Loaded {len(df)} SIP samples.")

# One-hot encode new data
df_encoded = pd.get_dummies(df)

# Align with training columns
df_encoded = df_encoded.reindex(columns=expected_cols, fill_value=0)

print(f"✅ Data aligned. Shape: {df_encoded.shape}")

# Predict covert traffic
predictions = model.predict(df_encoded)
df['covert_prediction'] = (predictions > 0.5).astype(int)

# Save results
df.to_csv("covert_alerts.csv", index=False)
print("✅ Detection complete. Saved to covert_alerts.csv")

# Optional: Show summary
covert_count = df['covert_prediction'].sum()
print(f"⚠️ Detected {covert_count} covert messages out of {len(df)} total.")


✅ Loaded 2536 SIP samples.
✅ Data aligned. Shape: (2536, 39349)
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
✅ Detection complete. Saved to covert_alerts.csv
⚠️ Detected 40 covert messages out of 2536 total.


In [ ]:
# Save full detection results
df.to_csv("detection_output.csv", index=False)
print("✅ Detection complete. Results saved to detection_output.csv")

# Filter only covert messages
covert_df = df[df['covert_prediction'] == 1]



✅ Detection complete. Results saved to detection_output.csv


In [ ]:
# Extract key metadata columns (customize if your CSV uses different column names)
key_fields = ['src_ip', 'dst_ip', 'Call-ID', 'From', 'To']
available_keys = [col for col in key_fields if col in covert_df.columns]

if not available_keys:
    print("⚠️ No key SIP fields like src_ip, dst_ip, from, to found in dataset.")
else:
    # Show detected covert messages with metadata
    print("🚨 Covert Messages Detected:")
    print(covert_df[available_keys + ['covert_prediction']].to_string(index=False))

    # Save for IDS rule updating
    covert_df[available_keys].to_csv("covert_alerts.csv", index=False)
    print("✅ Key SIP fields of covert detections saved to covert_alerts.csv")

🚨 Covert Messages Detected:
        src_ip        dst_ip                 Call-ID                                                                     From                                               To  covert_prediction
38.242.140.216 178.18.244.71  1-68818@38.242.140.216  From: 854081310 <sip:854081310@38.242.140.216:5080>;tag=68818SIPpTag001 To: 955954218 <sip:955954218@178.18.244.71:5080>                  1
38.242.140.216 178.18.244.71  2-68818@38.242.140.216  From: 301830044 <sip:301830044@38.242.140.216:5080>;tag=68818SIPpTag002 To: 177205718 <sip:177205718@178.18.244.71:5080>                  1
38.242.140.216 178.18.244.71  3-68818@38.242.140.216  From: 647053985 <sip:647053985@38.242.140.216:5080>;tag=68818SIPpTag003 To: 294823834 <sip:294823834@178.18.244.71:5080>                  1
38.242.140.216 178.18.244.71  4-68818@38.242.140.216  From: 459170909 <sip:459170909@38.242.140.216:5080>;tag=68818SIPpTag004 To: 291631055 <sip:291631055@178.18.244.71:5080>                  1
38